In [ ]:
# %pip install kagglehub


Note: you may need to restart the kernel to use updated packages.


In [2]:

import kagglehub
import shutil # 파일 옮겨주는 라이브러리

# 데이터 다운로드 
## 파일이 저장된 path 
data_path = kagglehub.dataset_download("thedevastator/2010-new-york-state-hospital-inpatient-discharge")
target_path = 'datasets'
shutil.move(data_path, target_path)

c:\Users\Playdata\AppData\Local\miniconda3\envs\ml\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 109M/109M [00:03<00:00, 31.7MB/s] 

Extracting files...


'datasets'

In [14]:
import pandas as pd
import numpy as np

# 데이터 불러오기 
data = pd.read_csv('datasets/hospital-inpatient-discharges-sparcs-de-identified-2010-1.csv')
data.head(10)
print(data.shape)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_25552\3902622579.py:5: DtypeWarning: Columns (1,2,7,11,29) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('datasets/hospital-inpatient-discharges-sparcs-de-identified-2010-1.csv')


(2622133, 38)


In [15]:
df = data[data['Length of Stay'].apply(lambda x: isinstance(x, int))]
print(df.shape)

(98997, 38)


In [21]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline

# data.columns
X_columns = df.drop(columns='Length of Stay').columns
cate_columns = ['Health Service Area', 'Hospital County', 'Facility Name', 'Age Group', 'Gender', 'Race', 'Ethnicity', 'Type of Admission', 'Patient Disposition', 'CCS Diagnosis Description', 'CCS Procedure Description', 'APR DRG Description', 'APR MDC Description', 'APR Medical Surgical Description', 'Source of Payment 1']
num_columns = ['Operating Certificate Number', 'Facility ID', 'Discharge Year', 'CCS Diagnosis Code', 'CCS Procedure Code', 'APR DRG Code', 'APR MDC Code', 'APR Severity of Illness Code']


# 범주형 처리 파이프라인
cat_pipeline = make_pipeline(
    SimpleImputer(strategy='most_frequent'),
    OneHotEncoder(handle_unknown='ignore')
)

# 수치형 처리 파이프라인
num_pipeline = make_pipeline(
    SimpleImputer(strategy='most_frequent'),  # most_frequent or mean
    StandardScaler()
)

# 전체 전처리기
preprocessor = ColumnTransformer([
    ('cat', cat_pipeline, cate_columns),
    ('num', num_pipeline, num_columns)
])

# 최종 파이프라인
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('rfr', RandomForestRegressor(n_estimators=500, max_depth=5))
])
pipeline

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Health Service Area',
                                                   'Hospital County',
                                                   'Facility Name', 'Age Group',
                                                   'Gender', 'Race',
                                                   'Ethnicity',
                                                   'Type of Admission',
                                                   'Patient Disposition',
                                                   'CCS Diagnosis De...
                                                   'Source of Payment 1']),
                                                 ('num',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Operating Certificate '
                                                   'Number',
                                                   'Facility ID',
                                                   'Discharge Year',
                                                   'CCS Diagnosis Code',
                                                   'CCS Procedure Code',
                                                   'APR DRG Code',
                                                   'APR MDC Code',
                                                   'APR Severity of Illness '
                                                   'Code'])])),
                ('rfr', RandomForestRegressor(max_depth=5, n_estimators=500))])

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
import numpy as np

######################
# 전처리
######################

print('='*25, '전처리 결과 확인', '='*25)
# 1. X, y 분리
X = df.drop(columns='Length of Stay')
y = df['Length of Stay']

# 2. 학습용/검증용 분리 (선택적)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. 파이프라인 학습
pipeline.fit(X_train, y_train)

# 4. 전처리기만 따로 사용해 변환된 X 추출
X_train_transformed = pipeline.named_steps['preprocessor'].transform(X_train)

# 5. 변환된 데이터 형태 확인
print("전처리된 X_train type:", type(X_train_transformed))
print("전처리된 X_train shape:", X_train_transformed.shape)

# 6. 희소행렬이면 밀집행렬로 변환
if hasattr(X_train_transformed, "toarray"):
    X_train_array = X_train_transformed.toarray()
else:
    X_train_array = X_train_transformed

# 7. feature 이름 추출 (OneHotEncoder 포함)
feature_names = pipeline.named_steps['preprocessor'].get_feature_names_out()

# 8. DataFrame으로 변환해 보기
X_train_df = pd.DataFrame(X_train_array, columns=feature_names)
print(X_train_df.head())

print()
print()
print()

#################
# 모델링
#################

print('='*25, '모델링 결과 확인', '='*25)
# 1. 데이터 분할
X = df.drop(columns='Length of Stay')
y = df['Length of Stay']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. 모델 학습
pipeline.fit(X_train, y_train)

# 3. 예측
y_pred = pipeline.predict(X_test)

# 4. 평가지표 계산
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# 5. 출력
print("✅ RandomForest 회귀 모델 성능 평가:")
print(f"MAE  (평균 절대 오차): {mae:.3f}")
print(f"MSE  (평균 제곱 오차): {mse:.3f}")
print(f"RMSE (평균 제곱근 오차): {rmse:.3f}")
print(f"R²   (설명력): {r2:.4f}")


========================= 전처리 결과 확인 =========================
전처리된 X_train type: <class 'scipy.sparse._csr.csr_matrix'>
전처리된 X_train shape: (79197, 891)
   cat__Health Service Area_Capital/Adiron  \
0                                      0.0   
1                                      0.0   
2                                      0.0   
3                                      0.0   
4                                      0.0   

   cat__Health Service Area_Central NY  cat__Health Service Area_Finger Lakes  \
0                                  0.0                                    0.0   
1                                  0.0                                    0.0   
2                                  0.0                                    0.0   
3                                  0.0                                    0.0   
4                                  0.0                                    0.0   

   cat__Health Service Area_Hudson Valley  \
0                                     